In [10]:
import cv2
import mediapipe as mp
import face_recognition
from IPython.display import display, Image, clear_output
from arango import ArangoClient

In [ ]:
client = ArangoClient(hosts="http://localhost:8529")

db = client.db("Shoptaki-recog", username="root", password="openSesame")

people = db.collection("people")
people.add_index({'type': 'persistent', 'fields': ['name'],})
people.insert({"name": "Gangey", "path": "photo.jpeg"})

In [7]:
def load_static_image_encoding(image_path):
    static_img = face_recognition.load_image_file(image_path)
    return face_recognition.face_encodings(static_img)[0]

def show_live_video():
    static_image_encoding = load_static_image_encoding("photo.jpeg")
    mp_face_detection = mp.solutions.face_detection
    mp_drawing = mp.solutions.drawing_utils

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Unable to access the webcam.")
        return

    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.2) as face_detection:
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    print("Error: Failed to capture image.")
                    break

                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                results = face_detection.process(frame_rgb)

                if results.detections:
                    for detection in results.detections:
                        # Extract bounding box information
                        h, w, _ = frame_rgb.shape
                        try:
                            top = int(detection.location_data.relative_bounding_box.ymin * h)
                            right = int((detection.location_data.relative_bounding_box.xmin + detection.location_data.relative_bounding_box.width) * w)
                            bottom = int((detection.location_data.relative_bounding_box.ymin + detection.location_data.relative_bounding_box.height) * h)
                            left = int(detection.location_data.relative_bounding_box.xmin * w)
                            face_roi = frame_rgb[top:bottom, left:right]

                            # Convert the face region to BGR color (required by face_recognition)
                            face_roi_bgr = cv2.cvtColor(face_roi, cv2.COLOR_RGB2BGR)

                            # Detect face locations and encodings in the face ROI
                            face_locations = face_recognition.face_locations(face_roi_bgr)
                            face_encodings = face_recognition.face_encodings(face_roi_bgr, face_locations)

                            if face_encodings:
                                face_encoding = face_encodings[0]
                                results = face_recognition.compare_faces([static_image_encoding], face_encoding)
                                if results[0]:
                                    print("Match found!")
                                    break
                                else:
                                    print("No match")

                            mp_drawing.draw_detection(frame, detection)
                            clear_output(wait=True)
                            display(Image(data=cv2.imencode('.jpg', frame)[1].tobytes()))
                        except AttributeError:
                            print("Error: Could not extract bounding box information.")
                            continue

        except KeyboardInterrupt:
            cap.release()
            cv2.destroyAllWindows()